In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Concatanate all tweets datasets

In [ ]:
import os
import glob
import pandas as pd
os.chdir("/content/drive/My Drive/243/FakeNewsNet/dataset/tweets")

In [ ]:
extension = 'csv'
all_filenames = ["/content/drive/My Drive/243/FakeNewsNet/dataset/tweets/" + i for i in glob.glob('*.{}'.format(extension))]

In [ ]:
len(all_filenames)

38

In [ ]:
combined_csv = pd.concat([pd.read_csv(f,lineterminator='\n').reset_index(drop=True) for f in all_filenames],axis=0,ignore_index=True)

In [ ]:
bool_series = pd.isnull(combined_csv["Quote"]) 
combined_csv['null'] = bool_series
for i in range(len(combined_csv)):
  if combined_csv['null'][i]:
    combined_csv['Quote'][i] = combined_csv['Quote\r'][i]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [ ]:
combined_csv

,Id,Tweet_Id,Datetime,Text,Retweet,Like,Reply,Quote,Quote\r,source,number_tweets,fake_news,null
0,2493749932,284329075902926848,2012-12-27 16:05:08+00:00,Did Miley Cyrus and Liam Hemsworth secretly ge...,0,0,0,0.0,NaN,NaN,NaN,NaN,False
1,2493749932,284332744559968256,2012-12-27 16:19:43+00:00,Did Miley Cyrus and Liam Hemsworth secretly ge...,0,0,0,0.0,NaN,NaN,NaN,NaN,False
2,2493749932,284335412590297089,2012-12-27 16:30:19+00:00,Did Miley Cyrus and Liam Hemsworth secretly ge...,0,0,0,0.0,NaN,NaN,NaN,NaN,False
3,2493749932,284359918792880128,2012-12-27 18:07:42+00:00,Afternoon Tea – Daily Link Roundup: Did Miley ...,0,0,0,0.0,NaN,NaN,NaN,NaN,False
4,2493749932,284385385151332353,2012-12-27 19:48:54+00:00,Did Miley Cyrus and Liam Hemsworth secretly ge...,0,0,0,0.0,NaN,NaN,NaN,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2824301,671,1071555101234475009,2018-12-09 00:00:19,Listen to this fantastic speech by Missouri St...,0,0,0,NaN,NaN,politifact,11714.0,1.0,True
2824302,671,1071555111435071488,2018-12-09 00:00:21,Listen to this fantastic speech by Missouri St...,1,1,0,NaN,NaN,politifact,11714.0,1.0,True
2824303,671,1071555140052754432,2018-12-09 00:00:28,Listen to this fantastic speech by Missouri St...,0,0,0,NaN,NaN,politifact,11714.0,1.0,True
2824304,671,1071566460634443776,2018-12-09 00:45:27,"The one deserving dues is this #columnist, @pe...",0,1,1,NaN,NaN,politifact,11714.0,1.0,True


In [ ]:
clean_csv = pd.DataFrame()
clean_csv = combined_csv.iloc[:,:7]

In [ ]:
clean_csv

,Id,Tweet_Id,Datetime,Text,Retweet,Like,Reply
0,2493749932,284329075902926848,2012-12-27 16:05:08+00:00,Did Miley Cyrus and Liam Hemsworth secretly ge...,0,0,0
1,2493749932,284332744559968256,2012-12-27 16:19:43+00:00,Did Miley Cyrus and Liam Hemsworth secretly ge...,0,0,0
2,2493749932,284335412590297089,2012-12-27 16:30:19+00:00,Did Miley Cyrus and Liam Hemsworth secretly ge...,0,0,0
3,2493749932,284359918792880128,2012-12-27 18:07:42+00:00,Afternoon Tea – Daily Link Roundup: Did Miley ...,0,0,0
4,2493749932,284385385151332353,2012-12-27 19:48:54+00:00,Did Miley Cyrus and Liam Hemsworth secretly ge...,0,0,0
...,...,...,...,...,...,...,...
2824301,671,1071555101234475009,2018-12-09 00:00:19,Listen to this fantastic speech by Missouri St...,0,0,0
2824302,671,1071555111435071488,2018-12-09 00:00:21,Listen to this fantastic speech by Missouri St...,1,1,0
2824303,671,1071555140052754432,2018-12-09 00:00:28,Listen to this fantastic speech by Missouri St...,0,0,0
2824304,671,1071566460634443776,2018-12-09 00:45:27,"The one deserving dues is this #columnist, @pe...",0,1,1


Add tweet hours and tweet days

In [ ]:
clean_csv['Datetime'] = clean_csv['Datetime'].apply(lambda x: x.replace('+00:00',''))

In [ ]:
import datetime
from datetime import datetime
clean_csv['Datetime'] = clean_csv['Datetime'].apply(lambda x: datetime.strptime(x,"%Y-%m-%d %H:%M:%S"))

In [ ]:
clean_csv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2824306 entries, 0 to 2824305
Data columns (total 7 columns):
 #   Column    Dtype         
---  ------    -----         
 0   Id        int64         
 1   Tweet_Id  int64         
 2   Datetime  datetime64[ns]
 3   Text      object        
 4   Retweet   int64         
 5   Like      int64         
 6   Reply     int64         
dtypes: datetime64[ns](1), int64(5), object(1)
memory usage: 150.8+ MB


In [ ]:
# Extract hour and weekday from tweet posting datetime
clean_csv['Hour'] = clean_csv['Datetime'].apply(lambda x: x.hour)
clean_csv['Weekday'] = clean_csv.Datetime.dt.dayofweek
clean_csv['Month'] = clean_csv.Datetime.dt.month

In [ ]:
clean_csv

,Id,Tweet_Id,Datetime,Text,Retweet,Like,Reply,Hour,Weekday,Month
0,2493749932,284329075902926848,2012-12-27 16:05:08,Did Miley Cyrus and Liam Hemsworth secretly ge...,0,0,0,16,3,12
1,2493749932,284332744559968256,2012-12-27 16:19:43,Did Miley Cyrus and Liam Hemsworth secretly ge...,0,0,0,16,3,12
2,2493749932,284335412590297089,2012-12-27 16:30:19,Did Miley Cyrus and Liam Hemsworth secretly ge...,0,0,0,16,3,12
3,2493749932,284359918792880128,2012-12-27 18:07:42,Afternoon Tea – Daily Link Roundup: Did Miley ...,0,0,0,18,3,12
4,2493749932,284385385151332353,2012-12-27 19:48:54,Did Miley Cyrus and Liam Hemsworth secretly ge...,0,0,0,19,3,12
...,...,...,...,...,...,...,...,...,...,...
2824301,671,1071555101234475009,2018-12-09 00:00:19,Listen to this fantastic speech by Missouri St...,0,0,0,0,6,12
2824302,671,1071555111435071488,2018-12-09 00:00:21,Listen to this fantastic speech by Missouri St...,1,1,0,0,6,12
2824303,671,1071555140052754432,2018-12-09 00:00:28,Listen to this fantastic speech by Missouri St...,0,0,0,0,6,12
2824304,671,1071566460634443776,2018-12-09 00:45:27,"The one deserving dues is this #columnist, @pe...",0,1,1,0,6,12


In [ ]:
clean_csv.drop_duplicates(subset=['Tweet_Id'],inplace=True)

In [ ]:
clean_csv

,Id,Tweet_Id,Datetime,Text,Retweet,Like,Reply,Hour,Weekday,Month
0,2493749932,284329075902926848,2012-12-27 16:05:08,Did Miley Cyrus and Liam Hemsworth secretly ge...,0,0,0,16,3,12
1,2493749932,284332744559968256,2012-12-27 16:19:43,Did Miley Cyrus and Liam Hemsworth secretly ge...,0,0,0,16,3,12
2,2493749932,284335412590297089,2012-12-27 16:30:19,Did Miley Cyrus and Liam Hemsworth secretly ge...,0,0,0,16,3,12
3,2493749932,284359918792880128,2012-12-27 18:07:42,Afternoon Tea – Daily Link Roundup: Did Miley ...,0,0,0,18,3,12
4,2493749932,284385385151332353,2012-12-27 19:48:54,Did Miley Cyrus and Liam Hemsworth secretly ge...,0,0,0,19,3,12
...,...,...,...,...,...,...,...,...,...,...
1571025,11208,1059493983548653568,2018-11-05 17:13:44,A comprehensive investigation of voter imperso...,0,0,0,17,0,11
1571026,11208,1059527997928157185,2018-11-05 19:28:54,From 2014: A comprehensive investigation of vo...,0,1,1,19,0,11
1571027,11208,1059541085398155265,2018-11-05 20:20:54,A comprehensive investigation of voter imperso...,0,0,0,20,0,11
1571028,11208,1060959628127653888,2018-11-09 18:17:41,Can we dust of this study to remind #trump the...,0,1,0,18,4,11


Group by and get average or mode

In [ ]:
group_tweet_1 = clean_csv.groupby('Id').agg(Retweet=('Retweet','sum'), Like= ('Like','sum'), Reply=('Reply','sum')).reset_index()

In [ ]:
group_tweet_2 = clean_csv.groupby('Id')['Hour','Weekday','Month'].agg(lambda x: pd.Series.mode(x)[0]).reset_index()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


In [ ]:
group_tweet_3 = clean_csv['Id'].value_counts().to_frame('tweets_count').reset_index()

In [ ]:
group_tweet_ = group_tweet_1.merge(group_tweet_2, on = "Id", how = 'inner')

In [ ]:
group_tweet = group_tweet_3.merge(group_tweet_, left_on = "index", right_on = 'Id', how = 'inner')

In [ ]:
group_tweet.drop(['index'], axis = 1, inplace = True)

In [ ]:
group_tweet

,tweets_count,Id,Retweet,Like,Reply,Hour,Weekday,Month
0,20684,14548,63810,145948,16299,12,2,7
1,20266,12104,64296,224194,13971,16,6,7
2,14226,10371,34397,79000,9804,15,6,7
3,10555,14667,20892,77096,3123,16,3,7
4,10296,13900,82814,214627,20294,18,6,7
...,...,...,...,...,...,...,...,...
19948,1,922151,0,0,0,12,5,3
19949,1,864636,0,0,0,0,0,7
19950,1,865014,0,0,0,19,2,7
19951,1,911057,0,0,0,16,5,2


Combined with main dataset

In [ ]:
df = pd.read_csv("/content/drive/My Drive/243/FakeNewsNet/dataset/dataframe_text_no_tweets_v2.csv")

In [ ]:
# only retain relevant columns for data analysis
df.drop(['Unnamed: 0'], axis = 1, inplace = True)

In [ ]:
merge_df = df.merge(group_tweet, left_on = "id", right_on = "Id", how = 'inner')

In [ ]:
merge_df.dropna(inplace = True)
merge_df.drop(['Id'], axis = 1, inplace = True)

In [ ]:
merge_df.rename(columns={"target": "fake_news"},inplace = True)
merge_df['fake_news'] = merge_df['fake_news'].astype(int)

In [ ]:
merge_df.drop(['number_tweets'], axis = 1, inplace = True)

In [ ]:
merge_df

,id,source,news_url,source_url,title,publish_date,body,tweet_ids,fake_news,tweets_count,Retweet,Like,Reply,Hour,Weekday,Month
0,2493749932,gossipcop,http://www.dailymail.co.uk/tvshowbiz/article-5...,dailymail,Did Miley Cyrus and Liam Hemsworth secretly ge...,2018-06-22,Congratulations might be in order for Miley Cy...,"['284329075902926848', '284332744559968256', '...",0,85,45,186,7,21,4,12
1,941805037,gossipcop,http://variety.com/2017/biz/news/tax-march-don...,variety,Celebrities Join Tax March in Protest of Donal...,2017-04-15,Thousands are taking the streets to protest Pr...,"['853359353532829696', '853359576543920128', '...",0,70,15,44,2,21,5,4
2,2547891536,gossipcop,http://www.dailymail.co.uk/femail/article-3499...,dailymail,Cindy Crawford's daughter Kaia Gerber wears a ...,2016-03-18,We'd venture to say that Cindy Crawford's daug...,"['988821905196158981', '988824206556172288', '...",0,12,6,11,2,17,1,4
3,5476631226,gossipcop,http://variety.com/2018/film/news/list-2018-os...,variety,Full List of 2018 Oscar Nominations – Variety,2018-01-23,Oscar nominations for the 90th annual awards w...,"['955792793632432131', '955795063925301249', '...",0,61,7,34,3,14,1,1
4,5189580095,gossipcop,http://www.townandcountrymag.com/society/tradi...,townandcountrymag,Here's What Really Happened When JFK Jr. Met P...,2017-06-29,"During the summer of 1995, John F. Kennedy Jr....","['890253005299351552', '890401381814870016', '...",0,4,5,6,0,2,3,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16379,4588,politifact,http://www.politifact.com/texas/statements/201...,politifact,Rick Perry says he switched to GOP at a younge...,2011-09-06,"On his Aug. 30 radio program, conservative com...","['111125963881975809', '123928429862191105', '...",1,5,8,2,0,1,2,10
16382,12052,politifact,https://www.washingtonpost.com/news/the-fix/wp...,washingtonpost,"The CNN Miami Republican debate transcript, an...",2016-03-10,"To see an annotation, click or tap the highlig...","['708094885899968512', '708096400601419776', '...",1,33,6,5,1,1,4,3
16383,8071,politifact,http://www.whitehouse.gov/the-press-office/201...,whitehouse,Remarks by the President on the Economy -- Kno...,2013-07-24,"Knox College\n\nGalesburg, Illinois\n\n12:13 P...","['359704596446330880', '360079198183886848', '...",1,5,3,0,1,16,2,7
16386,13619,politifact,http://www.cnn.com/2017/01/05/politics/border-...,cnn,"Trump asking Congress, not Mexico, to pay for ...",2017-01-05,Story highlights The Trump transition team say...,"['817357495047979008', '817357627566985217', '...",1,837,1074,1519,260,13,4,1


In [ ]:
merge_df.to_csv('/content/drive/My Drive/243/FakeNewsNet/dataset/merge_df.csv', index=False)